### Carlos Morán y Carlos Tardón grupo 9

# Práctica 1.B Problema 2

En el siguiente puzle infantil se pide rellenar los espacios en blanco con números del 1 al 9 de tal modo que la ecuación sea correcta siguiendo el orden de las operaciones. Los números no pueden repeetirse. Es decir, en cada uno de los 9 huecos en blanco deberá ir un número sin reptetirse. Según declaraciones del profesor Tran Phuong existen 362.880 combinaciones para colocar los 9 dígitos en loso 9 espacios en blanco, pero sólo unas pocas son correctas.

### Interpretación del enunciado

Una instancia válida de este problema consistirá en dos tuplas (una de n elementos para los números con los espacios en blanco incluídos y otra tupla de n-1 elementos representando las operaciones). Los atributos de este problema serán esas esas dos tuplas, el número de espacios en blanco y los números disponibles (representados mediante un entero m que indica que estarán disponibles los números del 1 al m).

### Representación de Estados

Una tupla incialmente vacía que va almacenando (por orden) los números que se van colocando en los espacios en blanco. 

### Representación de Acciones
Las acciones se representarán como números enteros del 1 al 9 indicando los números que aún no se han metido en nums.

### Heurística usada
La heurística que hemos probado consiste en hallar uno de los mayores números alcanzables. Esto lo conseguimos asignando los mayores números no usados a las operaciones '+' y '*' y los menores números no usados a las operaciones '-' y '/'. De esta manera obtenemos un "mayor número alcanzable" (no necesariamente el máximo pero una aproximación de éste) y, cuanto más pequeño sea este (más concretamente, cuanto más se aproxime al número buscado), mejor será esa rama particular del árbol.

In [1]:
from search import *

def opera(op1,sig,op2):
    if op1 == -1:
        return op2

    if op2 == -1:
        return op1

    if sig == '+':
        return op1 + op2
    elif sig == '-':
        return op1 - op2
    elif sig == '*':
        return op1 * op2
    else:
        return op1 / op2

In [42]:
class Ecuacion(Problem):

    def __init__(self, initial, n = 9):
        self.nums = initial[0]
        self.ops = initial[1]
        self.n = n # admitimos numeros del 1 al n
        self.analizados = 0
        self.blancos = 0
        for i in range(0,len(self.nums)):
            if self.nums[i] == -1:
                self.blancos += 1
        Problem.__init__(self,tuple(), None)

    def actions(self,estado):
        ## Nota: Este if existe para el caso en que el número de blancos sea menor que 9
        accs=list()
        if len(estado) < self.blancos:
            cjt1 = set(self.nums).union(estado)
            cjt2 = set(range(1,self.n + 1))
            cjt3 = cjt2.difference(cjt1)
            
            for i in cjt3:
                accs.append(i)
        
        return accs

    def result(self,estado,accion):
        nums = list(estado)
        nums.append(accion)
        self.analizados += 1
        return tuple(nums)

    ## Nota: Esto se hace suponiendo que solo hay un símbolo igual y que éste está al final 
    def goal_test(self,estado):
        if len(estado) < self.blancos:
            return False

        ind = 0

        if self.nums[0] == -1:
                res = estado[ind]
                ind += 1
        else: res = nums[0]

        for i in range(0,len(self.ops)-1):
            if self.nums[i+1] == -1:
                res = opera(res,self.ops[i],estado[ind])
                ind += 1
            else: res = opera(res,self.ops[i],self.nums[i+1])

        if res != self.nums[-1]:
            return False

        return True

    def elegir(self,sig,nums):
        cjt1 = nums
        cjt2 = set(range(1,self.n + 1))
        dif = tuple(cjt2.difference(cjt1))

        if sig == '+' or sig == '*':
            return dif[-1]
        else: return dif[0]

    def heuristica(self,node):
        nums = list(self.nums)
        ind = 0
        if nums[0] == -1 and ind >= len(node.state):
            nums[0] = self.elegir('+',set(nums))
        elif nums[0] == -1:
            nums[0] = node.state[ind]
            ind += 1

        res = nums[0]

        for i in range(0,len(self.ops)-1):
            if nums[i+1] == -1 and ind >= len(node.state):
                nums[i+1] = self.elegir(self.ops[i],set(nums).union(set(node.state)))
            elif nums[i+1] == -1:
                nums[i+1] = node.state[ind]
                ind += 1
            res = opera(res,self.ops[i],nums[i+1])
        
        return abs(res-self.nums[-1])
    

    def initial_check(self): # Comprobaciones iniciales, para no llamar a la busqueda si no es necesario
        ## Si la diferencia de tamaños entre números y operaciones es mayor que 1 
        ## (corresponde a las instancias con fallos de sintaxis)
        if len(self.nums) != len(self.ops)+1:
            return False

        ## Para evitar instancias otro tipo de instancias con fallos de sintaxis (instancias con más blancos que números            disponibles)
        if self.blancos > self.n:
            return False
        
        return True  

In [43]:
def resuelve_ecuacion(problem, algoritmo,h=None, print_sol=True):
    if problem.initial_check():
        if h: 
            res = algoritmo(problem,h)
        else:
            res = algoritmo(problem)
        if res is None:
            print("Instancia sin solución")
            print("Nodos analizados: {0}".format(problem.analizados))
        else:
            sol = res.solution()
            if print_sol: #Para calcular tiempos, no printeamos la solución
                print("Solución: {0}".format(sol))
                print("Nodos analizados: {0}".format(problem.analizados))
                if h:
                    print("Heurística: {0}".format(h.__name__))
                print("Algoritmo: {0}".format(algoritmo.__name__))
    else: 
        print("Problema sin solución")



In [32]:
I1 = Ecuacion(((-1,13,-1,-1,-1,12,-1,-1,11,-1,-1,-1,10,66),('+','*','/','+','+','*','-','-','+','*','/','-','=')))
resuelve_ecuacion(I1,breadth_first_tree_search, print_sol=True)

Solución: [1, 2, 4, 7, 5, 8, 3, 6, 9]
Nodos analizados: 986409
Algoritmo: breadth_first_tree_search


In [33]:
%%timeit
## La del enunciado. Varias soluciones posbles
I1 = Ecuacion(((-1,13,-1,-1,-1,12,-1,-1,11,-1,-1,-1,10,66),('+','*','/','+','+','*','-','-','+','*','/','-','=')))
resuelve_ecuacion(I1,breadth_first_tree_search, print_sol=False)

4.76 s ± 440 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
I1 = Ecuacion(((-1,13,-1,-1,-1,12,-1,-1,11,-1,-1,-1,10,66),('+','*','/','+','+','*','-','-','+','*','/','-','=')))
resuelve_ecuacion(I1,depth_first_graph_search, print_sol=True)

Solución: [9, 7, 2, 8, 4, 3, 6, 1, 5]
Nodos analizados: 23683
Algoritmo: depth_first_graph_search


In [40]:
%%timeit
I1 = Ecuacion(((-1,13,-1,-1,-1,12,-1,-1,11,-1,-1,-1,10,66),('+','*','/','+','+','*','-','-','+','*','/','-','=')))
resuelve_ecuacion(I1,depth_first_graph_search, print_sol=False)

357 ms ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
I1 = Ecuacion(((-1,13,-1,-1,-1,12,-1,-1,11,-1,-1,-1,10,66),('+','*','/','+','+','*','-','-','+','*','/','-','=')))
resuelve_ecuacion(I1,best_first_graph_search,I1.heuristica , print_sol=True)

Solución: [1, 2, 7, 5, 3, 4, 9, 8, 6]
Nodos analizados: 3332
Heurística: heuristica
Algoritmo: best_first_graph_search


In [45]:
%%timeit
I1 = Ecuacion(((-1,13,-1,-1,-1,12,-1,-1,11,-1,-1,-1,10,66),('+','*','/','+','+','*','-','-','+','*','/','-','=')))
resuelve_ecuacion(I1,astar_search,I1.heuristica , print_sol=False)

253 ms ± 39.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
I2 = Ecuacion(((-1,-1,-1,-1,-1,35),('+','+','+','+','=')))
resuelve_ecuacion(I2,breadth_first_tree_search, print_sol=True)

Solución: [5, 6, 7, 8, 9]
Nodos analizados: 18729
Algoritmo: breadth_first_tree_search


In [47]:
%%timeit
I2 = Ecuacion(((-1,-1,-1,-1,-1,35),('+','+','+','+','=')))
resuelve_ecuacion(I2,breadth_first_tree_search, print_sol=False)

80.3 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
I2 = Ecuacion(((-1,-1,-1,-1,-1,35),('+','+','+','+','=')))
resuelve_ecuacion(I2,depth_first_graph_search , print_sol=True)

Solución: [9, 8, 7, 6, 5]
Nodos analizados: 35
Algoritmo: depth_first_graph_search


In [49]:
%%timeit
I2 = Ecuacion(((-1,-1,-1,-1,-1,35),('+','+','+','+','=')))
resuelve_ecuacion(I2,depth_first_graph_search , print_sol=False)

305 µs ± 33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [50]:
I2 = Ecuacion(((-1,-1,-1,-1,-1,35),('+','+','+','+','=')))
resuelve_ecuacion(I2,best_first_graph_search,I2.heuristica , print_sol=True)

Solución: [5, 6, 7, 8, 9]
Nodos analizados: 35
Heurística: heuristica
Algoritmo: best_first_graph_search


In [51]:
%%timeit
I2 = Ecuacion(((-1,-1,-1,-1,-1,35),('+','+','+','+','=')))
resuelve_ecuacion(I2,best_first_graph_search,I2.heuristica , print_sol=False)

788 µs ± 227 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [52]:
## No hay solución posible
I3 = Ecuacion(((-1,-1,-1,-1,-1,36),('+','+','+','+','=')))
resuelve_ecuacion(I3,breadth_first_tree_search)

Instancia sin solución
Nodos analizados: 18729


In [53]:
I3 = Ecuacion(((-1,-1,-1,-1,-1,36),('+','+','+','+','=')))
resuelve_ecuacion(I3,best_first_graph_search,I3.heuristica)

Instancia sin solución
Nodos analizados: 18729


In [54]:
I3 = Ecuacion(((-1,-1,-1,-1,-1,36),('+','+','+','+','=')))
resuelve_ecuacion(I3,astar_search,I3.heuristica)

Instancia sin solución
Nodos analizados: 18729


In [56]:
## No hay solución posible (la diferencia entre los tamaños de nums y ops es distinta de 1 en valor absoluto)
I5 = Ecuacion(((-1,-1,-1,-1,-1,-1,-1,-1,-1,36),('+','+','+','+','+','+','+','+','+','=')))
resuelve_ecuacion(I5,breadth_first_tree_search)

Problema sin solución


In [57]:
I6 = Ecuacion(((-1,-1,-1,-1,-1,15),('+','+','+','+','=')))
resuelve_ecuacion(I6,breadth_first_tree_search , print_sol=True)

Solución: [1, 2, 3, 4, 5]
Nodos analizados: 18729
Algoritmo: breadth_first_tree_search


In [58]:
%%timeit
## Una solucíón posible: 1+2+3+4.
I6 = Ecuacion(((-1,-1,-1,-1,-1,15),('+','+','+','+','=')))
resuelve_ecuacion(I6,breadth_first_tree_search , print_sol=False)

64.1 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
I6 = Ecuacion(((-1,-1,-1,-1,-1,15),('+','+','+','+','=')))
resuelve_ecuacion(I6,depth_first_graph_search, print_sol=True)

Solución: [5, 4, 3, 2, 1]
Nodos analizados: 9555
Algoritmo: depth_first_graph_search


In [60]:
%%timeit
I6 = Ecuacion(((-1,-1,-1,-1,-1,15),('+','+','+','+','=')))
resuelve_ecuacion(I6,depth_first_graph_search, print_sol=False)

137 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
I6 = Ecuacion(((-1,-1,-1,-1,-1,15),('+','+','+','+','=')))
resuelve_ecuacion(I6,best_first_graph_search,I6.heuristica)

Solución: [1, 2, 3, 4, 5]
Nodos analizados: 35
Heurística: heuristica
Algoritmo: best_first_graph_search


### Comentarios sobre la heurística y la solución

En este problema se muestra claramente la importancia de obtener una heurística, pese a no ser admisible, pues la finalidad de este problema es encontrar una solución. Con búsqueda en anchura, en la instancia I1(la que nos daban de prueba en el pdf), se analizan casi 1 millon de nodos antes de llegar a la solución. Con el incremento de potencias de procesamiento, este caso es tratable. Sin embargo, si generalizaramos el problema (por ejemplo, admitiendo los números del 1 al n), no podríamos usar busqueda en anchura en un tiempo razonable. Como se ve con I1, la heurística consigue bajar los nodos analizados, de casi 1 millon a tan solo 3332.  
En la siguiente celda, comprobamos la efectividad de la heuristica con n=11, es decir, permitiendo los numeros del 1 al 11:

In [ ]:
# Ha estado hora y media y no ha terminado. Esto es debido al gran factor de ramificacion, que produce que haya 
# 11! = 39.916.800 hojas (estados finales)
I4 = Ecuacion(((-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,36),('+','+','+','+','-','+','+','-','+','=')),n=11)
resuelve_ecuacion(I4,breadth_first_tree_search)

In [63]:
# Tan solo analiza 2635 nodos para llegar a a la solucion.
I4 = Ecuacion(((-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,36),('+','+','+','+','-','+','+','-','+','=')),n=11)
resuelve_ecuacion(I4,best_first_graph_search,I6.heuristica)

Solución: [1, 2, 3, 4, 7, 5, 9, 10, 6, 11]
Nodos analizados: 2635
Heurística: heuristica
Algoritmo: best_first_graph_search


### Comentarios sobre la solución
<ol>
<li>Durante la solución del problema, podríamos llevar el resultado parcial de los números ya añadidos (para hacer más eficiente la ejecución de goal_test). Sin embargo, como tenemos que anotar qué números hemos usado y si quedan blancos o no (lo que es igual de eficiente en espacio), no llevamos este resultado parcial (es igual de eficiente en memoria, pese a la ganancia de eficiencia en tiempo). </li>
<li>Combiene añadir que, como en este problema no teenemos ningún coste asociado/criterio de optimalidad, una heurística solo puede servir para llegar más rápido a una solución y no para mejorar una posible solución.</li>
<li> Mejor algoritmo: como se observa, la heurística mejora notablemente los nodos analizados, y por tanto se puede usar búsqueda voraz para obtener una solución.</li>
</ol>

### Posibles complicaciones del problema
<ol>
<li>Admitir números negativos (tanto en blancos como por defecto en nums): Habría que cambiar la identificación de los blancos (por el caracter 'b' por ejemplo)</li>
<li>Ampliar los números con los que podemos rellenar los blancos de range(1,10) a range(n,m+1) con n <= m: Habría que modificar la función "elegir" (introducir n y m como parámetros) para que escoja el conjunto range(n,m+1) en vez de range(1,10)</li>
<li>Si podemos colocar range(n,m+1) elementos en los espacios en blanco, que en nums vengan "por defecto" números que estén en range(n,m+1): En vez de hacer diferencia de conjuntos en este caso, tendríamos que llevar en cada estado una tupla de booleanos (llamémosla usados) de tamaño m-n+1 inicializada en principio a false donde usados_i representa si ya se ha usado el número n+i (True) o no (False).</li>
<li>Blancos y/o operaciones a la derecha del igual: habría que modificar goal_test y la función de la heurística. Tendríamos que guardar en 2 variables distintas los resultados a la izquierda y a la derecha del igual y sería necesario recorrer nums hasta el igual primero y luego del igual en adelante. Otra opción sería preprocesar la ecuación para despejar todas las incógnitas, y después usar la resolución ya implementada</li>
<li>Más de un símbolo igual: La solución sería una versión iterada del apartado anterior con un pequeño detalle adicional, tenemos que saber previamente (como atributo adicional del problema) el número de símbolos igual. Sea ahora i la cantidad de símbolos de igual. Habría que modificar goal_test y la heurística y guardar en una tupla de i+1 elementos los diferentes resultados. Está claro que para calcular cada uno de esos resultados parciales necesitamos recorrer ese fragmento de la lista (vemos que el caso anterior sería un caso particular de este para i=1).</li>
</ol>